リアルデータでGLM

In [1]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nltools.file_reader import onsets_to_dm
from nltools.stats import regress, zscore
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nilearn.plotting import view_img, glass_brain, plot_stat_map
from bids import BIDSLayout, BIDSValidator

data_dir = '/Users/hidekiizumi/myproject/Localizer'
layout = BIDSLayout(data_dir, derivatives=True)

/Users/hidekiizumi/miniforge3/lib/python3.9/site-packages/bids/layout/validation.py:153: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


前処理の例で紹介したPinel Localizerデータセットを引き続き扱うことにする。

課題設定：
- 脳のどこの領域が顔を処理するのに関わっているかを特定する
- 顔の画像を見せた時に、脳のどの領域が活発化するかを調べる
  - trial : 1つの顔の画像を見せる実験
- 5 trialsでどのようになるかみてみる
- はじめに、facesを、タイムポイントの時系列ベクトルとしてして定義する
  - 全単位時間：200
  - 全トライアル回数：5

データセット: Pinel Localizer
目的: 視覚知覚、指のタッピング、言語、数学など、さまざまな基本的な認知プロセスを調査する
課題内容: 画面上のテキストを読む（視覚モーダリティ）および音声指示を聞く（聴覚モーダリティ）ことで、複数のタスクが提示される
実験デザイン: 条件をランダムに並べ替え、迅速なイベント関連デザインの効率を最大化するために最適化されている
実験時間: 5分のスキャンセッションで合計100試行
データセットの詳細: タスクおよびデータセットに関する詳細は、論文およびデータセット論文を参照
データセットの特徴: (a) 世界中の誰でも利用できる公開データセット, (b) 比較的小規模（約5分間のデータ）, (c) 異なるタイプの対照を作成するための多くのオプションがある
使用する被験者数: 全体で94人の被験者が利用可能であるが、主に10〜20人の小規模なサブセットを使用する

Pinel Localizerタスクは、視覚認識、指のタッピング、言語、数学など、いくつかの異なるタイプの基本的な認知プロセスを調査するように設計されています。いくつかの課題は、画面上のテキストを読むこと（すなわち視覚モダリティ）、また聴覚的な指示を聞くこと（すなわち聴覚モダリティ）を手がかりとしている。試行は条件によってランダムに行われ、迅速なイベント関連設計のために効率を最大化するように最適化されている。5分間のスキャニングセッションで合計100のトライアルがある。タスクの具体的な詳細については、オリジナルの[論文](https://bmcneurosci.biomedcentral.com/articles/10.1186/1471-2202-8-91)を、[データセット論文](https://doi.org/10.1016/j.neuroimage.2015.09.052)を読んでください。

このデータセットは、(a)世界中の誰でも利用可能で、(b)比較的小さく（5分程度）、(c)様々な種類のコントラストを作成するための多くのオプションを提供しているので、このチュートリアルに適しています。

全部で94の被験者が用意されていますが、私たちは主に10～20人の小さなサブセットだけを扱うことになります。Psych60版を受講していない方は、データのダウンロード方法についてチュートリアルをご覧ください。